In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd

from multiprocessing import Process
from CoolProp.CoolProp import PropsSI

import matplotlib.pyplot as plt
%matplotlib inline

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(norm(join(cwd, '..', '..', '..', 'standalone')))

from standalone.run_g_function import RunGFunctions

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('MFRTRT_loads.csv', parse_dates=True, index_col='Date/Time')

In [ ]:
df.head(3)

In [ ]:
df = df.resample('2T').mean()
df = df.interpolate(method='linear')

In [ ]:
df.head(3)

In [ ]:
df.plot(y=['mdot [kg/s]'])

In [ ]:
df.plot(y=['Meas. Total Power [W]'])

In [ ]:
df.to_csv('in.csv', columns=['Meas. Total Power [W]', 'mdot [kg/s]'])

In [ ]:
in_file = join(cwd, 'MFRTRT_input.json')
with open(in_file, 'r') as f:
    json_blob = f.read()
in_json = json.loads(json_blob)

In [ ]:
load_file = in_json['load-profile']['external']['path'] 
in_json['load-profile']['external']['path'] = join(cwd, load_file)

flow_file = in_json['flow-profile']['external']['path']
in_json['flow-profile']['external']['path'] = join(cwd, flow_file)

g_file = in_json['g-functions']['file']
in_json['g-functions']['file'] = join(cwd, g_file)

In [ ]:
in_json['load-aggregation']['type'] = 'dynamic'
in_json['simulation']['output-path'] = join(cwd, 'out_dynamic.csv')

with open(join(cwd, 'in_dynamic.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_json['load-aggregation']['type'] = 'static'
in_json['simulation']['output-path'] = join(cwd, 'out_static.csv')

with open(join(cwd, 'in_static.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_json['load-aggregation']['type'] = 'none'
in_json['simulation']['output-path'] = join(cwd, 'out_none.csv')

with open(join(cwd, 'in_none.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_static = os.path.join(os.getcwd(), 'in_static.json')
in_dynamic = os.path.join(os.getcwd(), 'in_dynamic.json')
in_none = os.path.join(os.getcwd(), 'in_none.json')

In [ ]:
RunGFunctions(in_static).simulate()

In [ ]:
RunGFunctions(in_dynamic).simulate()

In [ ]:
RunGFunctions(in_none).simulate()

In [ ]:
df_static = pd.read_csv("out_static.csv")

In [ ]:
df_dynamic = pd.read_csv("out_dynamic.csv")

In [ ]:
df_none = pd.read_csv("out_none.csv")

In [ ]:
start_pt = 1350
end_pt = 1500

x = df_static.index[start_pt : end_pt]

in_static = df_static['GLHE Inlet Temperature [C]'].values[start_pt : end_pt]
out_static = df_static['GLHE Outlet Temperature [C]'].values[start_pt : end_pt]
plt.plot(x, in_static, label='In (Static)', linestyle='--')
plt.plot(x, out_static, label='Out (Static)', linestyle='--')

in_dynamic = df_dynamic['GLHE Inlet Temperature [C]'].values[start_pt : end_pt]
out_dynamic = df_dynamic['GLHE Outlet Temperature [C]'].values[start_pt : end_pt]
plt.plot(x, in_dynamic, label='In (Dynamic)', linestyle='-.')
plt.plot(x, out_dynamic, label='Out (Dynamic)', linestyle='-.')

in_none = df_none['GLHE Inlet Temperature [C]'].values[start_pt : end_pt]
out_none = df_none['GLHE Outlet Temperature [C]'].values[start_pt : end_pt]
plt.plot(x, in_none, label='In (None)', linestyle='-.')
plt.plot(x, out_none, label='Out (None)', linestyle='-.')

in_exp = df['Inlet 1 [C]'].values[start_pt : end_pt]
out_exp = df['Outlet 1 [C]'].values[start_pt : end_pt]
plt.plot(x, in_exp, label='In (Exp)')
plt.plot(x, out_exp, label='Out (Exp)')

plt.legend()

plt.show()

In [ ]:
plt.plot(x, df_static['Flow Fraction [-]'][start_pt : end_pt], label='Static')
plt.plot(x, df_dynamic['Flow Fraction [-]'][start_pt : end_pt], label='Dynamic')
plt.plot(x, df_none['Flow Fraction [-]'][start_pt : end_pt], label='None')
plt.legend()
plt.show()

In [ ]:
plt.plot(x, df['Meas. Total Power [W]'].values[start_pt : end_pt], label="Exp Data")
plt.plot(x, df_static['GLHE Heat Transfer Rate [W]'].values[start_pt : end_pt], label="Static")
plt.plot(x, df_dynamic['GLHE Heat Transfer Rate [W]'].values[start_pt : end_pt], label="Dynamic")
plt.plot(x, df_none['GLHE Heat Transfer Rate [W]'].values[start_pt : end_pt], label="None")

plt.legend()
plt.show()

In [ ]:
plt.plot(x, df_static['Load on GHE [W/m]'].values[start_pt : end_pt], label='Static')
plt.plot(x, df_dynamic['Load on GHE [W/m]'].values[start_pt : end_pt], label='Dynamic') 
plt.plot(x, df_none['Load on GHE [W/m]'].values[start_pt : end_pt], label='None') 

plt.legend()
plt.show()

In [ ]:
plt.plot(x, df_static['Average Fluid Temp [C]'].values[start_pt : end_pt], label="Static")
plt.plot(x, df_dynamic['Average Fluid Temp [C]'].values[start_pt : end_pt], label="Dynamic")
plt.plot(x, df_none['Average Fluid Temp [C]'].values[start_pt : end_pt], label="None")

plt.legend()
plt.show()

In [ ]:
plt.plot(x, df_static["Local Borehole Resistance 'Rb' [K/(W/m)]"].values[start_pt : end_pt], label="Static")
plt.plot(x, df_dynamic["Local Borehole Resistance 'Rb' [K/(W/m)]"].values[start_pt : end_pt], label="Dynamic")
plt.plot(x, df_none["Local Borehole Resistance 'Rb' [K/(W/m)]"].values[start_pt : end_pt], label="None")

plt.legend()
plt.show()

In [ ]:
plt.plot(x, df_static["Soil Resistance 'Rs' [K/(W/m)]"].values[start_pt : end_pt], label="Static")
plt.plot(x, df_dynamic["Soil Resistance 'Rs' [K/(W/m)]"].values[start_pt : end_pt], label="Dynamic")
plt.plot(x, df_none["Soil Resistance 'Rs' [K/(W/m)]"].values[start_pt : end_pt], label="None")

plt.legend()
plt.show()